In [1]:
from io import BytesIO
import zipfile

import ipywidgets
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [2]:
def load_data(year):
    url = 'https://www.bls.gov/oes/special.requests/oesm{}nat.zip'.format(year % 100)
    resp = requests.get(url)
    zf = zipfile.ZipFile(BytesIO(resp.content))
    for fi in zf.filelist:
        if fi.filename.rstrip('x').endswith('_dl.xls'):
            f = zf.read(fi)
            df = pd.read_excel(f)
            df.columns = df.columns.str.lower()
            df['year'] = year
            return df
    raise RuntimeError('Could not find correct Excel file within Zip archive.')

In [3]:
def load_years(years):
    frames = []
    for year in years:
        df = load_data(year)
        frames.append(df)
    return frames

In [31]:
df_all.area_type.nunique()

1

In [25]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13208 entries, 0 to 1328
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   occ_code      13208 non-null  object 
 1   occ_title     13208 non-null  object 
 2   group         23 non-null     object 
 3   tot_emp       13208 non-null  object 
 4   emp_prse      13208 non-null  object 
 5   h_mean        13208 non-null  object 
 6   a_mean        13208 non-null  object 
 7   mean_prse     13208 non-null  float64
 8   h_pct10       13208 non-null  object 
 9   h_pct25       13208 non-null  object 
 10  h_median      13208 non-null  object 
 11  h_pct75       13208 non-null  object 
 12  h_pct90       13208 non-null  object 
 13  a_pct10       13208 non-null  object 
 14  a_pct25       13208 non-null  object 
 15  a_median      13208 non-null  object 
 16  a_pct75       13208 non-null  object 
 17  a_pct90       13208 non-null  object 
 18  annual        795 non-null 

In [4]:
df_all = pd.concat(load_years(range(2011, 2021)))
print(df_all.describe(include='all').to_latex())

\begin{tabular}{llllrrllrllllllllllllrlrlrrllrlrrrl}
\toprule
{} & occ\_code &               occ\_title &  group &  tot\_emp &  emp\_prse & h\_mean & a\_mean &     mean\_prse & h\_pct10 & h\_pct25 & h\_median & h\_pct75 & h\_pct90 & a\_pct10 & a\_pct25 & a\_median & a\_pct75 & a\_pct90 & annual & hourly &          year & occ\_group &    area & area\_title &  area\_type &   naics &     naics\_title &         i\_group &  own\_code &   o\_group &  jobs\_1000 &  loc\_quotient &  pct\_total & prim\_state \\
\midrule
count  &    13208 &                   13208 &     23 &  13208.0 &   13208.0 &  13208 &  13208 &  13208.000000 &   13208 &   13208 &    13208 &   13208 &   13208 &   13208 &   13208 &    13208 &   13208 &   13208 &    795 &     58 &  13208.000000 &      9731 &  2658.0 &       2658 &     2658.0 &  2658.0 &            2658 &            2658 &    2658.0 &      2658 &        0.0 &           0.0 &        0.0 &       1329 \\
unique &     1507 &                    1272 &      2 &   8454

In [5]:
# for i in df_all.occ_title:
#     print(i)

In [6]:
df_all.occ_title.value_counts()

Tour and Travel Guides                                              22
Carpenters                                                          19
Septic Tank Servicers and Sewer Pipe Cleaners                       19
Rock Splitters, Quarry                                              19
Podiatrists                                                         19
                                                                    ..
Protective Service Workers, All Other *                              1
Business Operations Specialists, All Other*                          1
Funeral Service Managers, Directors, Morticians, and Undertakers     1
Community and Social Service Specialists, All Other*                 1
Paralegals and Legal Assistants*                                     1
Name: occ_title, Length: 1272, dtype: int64

In [7]:
df_all['mean salary'] = pd.to_numeric(df_all['a_mean'], errors='coerce')

In [35]:
# major_groups = df_all['occ_code'].str.endswith('0000') # endswith 0000 is the mother node? 
major_groups = df_all['occ_code']
df_all['occ_code'].unique()


array(['00-0000', '11-0000', '11-1011', ..., '53-4022', '53-6098',
       '53-7065'], dtype=object)

In [34]:
df_all['occ_code'].unique()

array(['00-0000', '11-0000', '11-1011', ..., '53-4022', '53-6098',
       '53-7065'], dtype=object)

In [9]:
# sns.lineplot(data=df_all[major_groups], x='year', y='mean salary', hue='occ_title')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2)

In [21]:
# titles = sorted(list(df_all[major_groups]['occ_title'].unique()))
titles = sorted(list(df_all['occ_title'].unique()))


In [22]:
@ipywidgets.interact(title=titles)
def plot_group(title=titles[0]):
    ind = titles.index(title)
    for t in titles[:ind] + titles[ind+1:] + [title]:
        color = ('r' if t == title else '0.7')
        df = df_all[df_all['occ_title'] == t]
        plt.plot(df['year'], df['mean salary'], c=color)
    plt.xlabel('year')
    plt.ylabel('mean salary')

interactive(children=(Dropdown(description='title', options=('Accountants and Auditors', 'Actors', 'Actors, Pr…

# TBD

In [12]:
# import requests
# import json
# import prettytable
# headers = {'Content-type': 'application/json'}
# data = json.dumps({"seriesid": ['WMU00000001020000001730272400'],"startyear":"2020", "endyear":"2020"})
# p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
# json_data = json.loads(p.text)
# # for series in json_data['Results']['series']:
# #     x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
# #     seriesId = series['seriesID']
# #     for item in series['data']:
# #         year = item['year']
# #         period = item['period']
# #         value = item['value']
# #         footnotes=""
# #         for footnote in item['footnotes']:
# #             if footnote:
# #                 footnotes = footnotes + footnote['text'] + ','
# #         if 'M01' <= period <= 'M12':
# #             x.add_row([seriesId,year,period,value,footnotes[0:-1]])
# #     output = open(seriesId + '.txt','w')
# #     output.write (x.get_string())
# #     output.close()